In [1]:
import tigramite 
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, GridspecLayout
from IPython.display import display
from io import BytesIO

#tigramite imports

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn

import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

First very basic version

In [2]:
def calculateResults(Data, Mask, Method, Test):
    with terminal_out:
        data = np.load(Data)
        if (Mask != "none"):
            mask = np.load(Mask)
            dataframe = pp.DataFrame(data, mask=mask)
        else:
            dataframe = pp.DataFrame(data)
        cond_ind_test = getCondIndTest(Test)
        global pcmci
        pcmci = PCMCI(
            dataframe=dataframe, 
            cond_ind_test=cond_ind_test,
            verbosity=1)
        global results 
        if (Method=="PCMCI"):
            results = pcmci.run_pcmci(tau_max=8, pc_alpha=None)
        elif (Method == "PCMCI+"):
            results = pcmci.run_pcmciplus()
    return True

In [3]:
def makePlot(plot_type):
    try:
        q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
        var_names = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']
        """pcmci.print_significant_links(
                p_matrix = results['p_matrix'], 
                q_matrix = q_matrix,
                val_matrix = results['val_matrix'],
                alpha_level = 0.01)"""
        link_matrix = pcmci.return_significant_links(pq_matrix=q_matrix,
                        val_matrix=results['val_matrix'], alpha_level=0.01)['link_matrix']
        if plot_type=="Process Graph":
            tp.plot_graph(
                val_matrix=results['val_matrix'],
                link_matrix=link_matrix,
                var_names=var_names,
                link_colorbar_label='cross-MCI',
                node_colorbar_label='auto-MCI',
            )
        elif plot_type=="Time series graph":
            tp.plot_time_series_graph(
                figsize=(6, 4),
                val_matrix=results['val_matrix'],
                link_matrix=link_matrix,
                var_names=var_names,
                link_colorbar_label='MCI',
                )
        elif plot_type=="Lagged Correlation":
            correlations = pcmci.get_lagged_dependencies(tau_max=20, val_only=True)['val_matrix']
            lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                                    'x_base':5, 'y_base':.5})
        else: 
            print("This should not be possible!")
        with plot_out:
            plt.show()
    except Exception as e:
        print("Something went wrong here! Try executing the first part")
        print(e.message)
            
    return True
    

In [4]:
def getCondIndTest(Test):
    print(Test)
    if (Test == "ParCorr"):
        return ParCorr(significance='analytic')
    elif (Test == "GPDC"):
        return GPDC()
    elif (Test == "CMIknn"):
        return CMIknn()
    elif (Test == "CMIsymb"):
        return CMIsymb()
    else:
        raise Exception("Something is not right here.")

In [5]:
# widgets
print(os.getcwd())
listDir = []
for entry in os.scandir(os.getcwd()):
    listDir.append(entry.name)
listDir.append("none")
example_widget = widgets.Dropdown(
    options=listDir,#os.scandir(os.getcwd()),#.append("none"),
    value='none',
    description='Number:',
    disabled=False,
)
data_upload_widget = widgets.Dropdown(
    options=listDir,#os.scandir(os.getcwd()),#.append("none"),
    value='none',
    description='Data:',
    disabled=False
)

mask_upload_widget = widgets.Dropdown(
    options=listDir,#os.scandir(os.getcwd()),#.append("none"),
    value='none',
    description='Mask:',
    disabled=False,
)
method_widget = widgets.Dropdown(
    options=["PCMCI", "PCMCI+"],
    value="PCMCI",
    description="Method:",
    disabled=False
)
test_widget = widgets.Dropdown(
    options=["ParCorr", "GPDC", "CMIknn", "CMIsymb", "OracleCI"],
    value="ParCorr",
    description="CI Test:",
    disabled=False
)
add_method_parameter_button = widgets.Button(
    description = "+"
)

plots_widget = widgets.Dropdown(
    options=["Process Graph", "Time series graph", "Lagged Correlation"],
    value="Process Graph",
    description="Plot:",
    disabled=False,
    continuous_update=False,
    tooltip="This is the tip"
)

#only for .runpcmci()
method_parameter_widget = widgets.Dropdown(
    description = "Parameter",
    options=["selected_links", "tau_min", "tau_max", "save_iterations", "pc_alpha", "max_conds_dim", "max_combinations", "max_conds_py", "max_conds_px", "fdr_method"],
)

max_tau = widgets.IntSlider(
)

def getParameterValues(t): 
    if (t== "Method"):
        if (method_widget.value == "PCMCI"):
            return method_parameter_widget

parameters_widget = VBox(
    children= [HBox([getParameterValues("Method"), add_method_parameter_button])]
)
parameter_accordion = widgets.Accordion(
    children= [parameters_widget]
)
parameter_accordion.set_title(0, "Parameter Selection")
        
accordion = widgets.Accordion(children=[data_upload_widget, mask_upload_widget, VBox([method_widget, parameter_accordion]), test_widget])
accordion.set_title(2, 'Method')
accordion.set_title(3, 'Test')
accordion.set_title(0, 'Data')
accordion.set_title(1, 'Mask')

plot_out = widgets.Output(layout={'border': '1px solid black'})
terminal_out = widgets.Output(
    layout={'border': '1px solid black'},
)
run_button = widgets.Button(description="Run")

def on_button_clicked(b):
    return calculateResults(data_upload_widget.value, mask_upload_widget.value, method_widget.value, test_widget.value)

run_button.on_click(on_button_clicked)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])
        makePlot(change['new'])

def on_click_method_parameter(b):
    parameters_widget.children = tuple(list(parameters_widget.children) + [max_tau])
    print("tada")
add_method_parameter_button.on_click(on_click_method_parameter)
def clear_all():
    terminal_out.clear_output()
    plot_out.clear_output()
clear_all()
plots_widget.observe(on_change)
clear_all()

C:\Users\rosem\PycharmProjects\PrototypeTigramiteGui


In [ ]:
addInt = lambda x: x+1
addInt(3)

In [ ]:
def generateLayout(header, left, right, footer):
    AppLayout(header=header,
             left_sidebar=left,
             right_sidebar=right,
             footer=footer)

In [ ]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

footer_button.on_click(reload)


def reload(b):
    print("reloaded")
    return generateLayout(header_button, VBox([data_upload_widget, mask_upload_widget, HBox([method_widget, method_parameter_button, accordion]), test_widget, run_button]),VBox([plots_widget]),footer_button)

"""AppLayout(header=None,
          left_sidebar= VBox([data_upload_widget, mask_upload_widget, HBox([method_widget, method_parameter_button]), test_widget, run_button]),
          center= out,
          right_sidebar=VBox([plots_widget]),
          footer=None)"""

In [ ]:
grid = GridspecLayout(4,4)
grid[:2,:3] = accordion#VBox([data_upload_widget, mask_upload_widget, HBox([method_widget, method_parameter_button]), test_widget, run_button, plots_widget])
grid[:1,3:] = run_button
grid[2:,:1] = terminal_out
grid[2:, 1:] = plot_out
grid[1:2, 3:]= plots_widget


In [ ]:

w = VBox([accordion, run_button, plots_widget, HBox([terminal_out, plot_out])])
display(w)

Alte Version

In [ ]:

var_names = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']
@interact_manual
def calcInput(Data=os.listdir(), Mask=example_widget, Method=["PCMCI", "PCMCI+"], Tests=["Test1", "Test2"]):
    return calculateResults(Data, Mask, Method, Tests)

@interact_manual
def showPlots(plot_type=["Process Graph", "Time series graph", "Lagged Correlation"]):
    return makePlot(plot_type.value)

In [1]:
from PrototypeWidgets import ParameterSelectionWidget

In [2]:
values = [ "tau_min","selected_links", "tau_max", "save_iterations", "pc_alpha", "max_conds_dim", "max_combinations", "max_conds_py", "max_conds_px", "fdr_method"]
value_types = ["int", "dict",  "int", "bool", "float", "int", "int", "int", "int", "str"]
values_default = ["0", "None", "1", "False", "0.5", "None", "1", "None", "None", "None"]
test = ParameterSelectionWidget(values, value_types, values_default)
test.show()

Accordion(children=(VBox(children=(HBox(children=(Dropdown(options=('tau_min', 'selected_links', 'tau_max', 's…

Parameter tau_minint0
bla


In [ ]:
class parameter:
    def __init__(name, t, d):
        self.name = name
        self.t = t
        self.d = d
    

In [ ]:
values_dict = {
    "selected_links", 
    "tau_min", 
    "tau_max": "int", 
    "save_iterations": "bool", 
    "pc_alpha": "float", 
    "max_conds_dim": "int", 
    "max_combinations": "int", 
    "max_conds_py": "int", 
    "max_conds_px": "int", 
    "fdr_method": "str",
}

In [ ]:
test.get_widget(test)

In [ ]:
parameters_widget = widgets.VBox(
     15             children=[widgets.HBox([widgets.Dropdown(options=values), self.add_parameter_button])]
     16         )

In [2]:
import PrototypeWidgets.widgets as pwidgets
import os
pwidgets.accordionWidget(os.getcwd())

In [1]:
import PrototypeWidgets.selectionWindow as sw
sw.selection_window()

{'name': 'selected_links', 'dtype': 'dict', 'default': 'None'}
{'name': 'selected_links', 'dtype': 'dict', 'default': 'None'}
{'name': 'selected_links', 'dtype': 'dict', 'default': 'None'}
